In [2]:
from pyscf import gto, scf, mp, cc

a = 2 # bond length in a cluster
d = 3 # distance between each cluster
unit = 'b' # unit of length
na = 10  # size of a cluster (monomer)
nc = 1 # set as integer multiple of monomers
spin = 1 # spin per monomer
frozen = 0 # frozen orbital per monomer
elmt = 'H'
basis = 'sto6g'
atoms = ""
for n in range(nc*na):
    shift = ((n - n % na) // na) * (d-a)
    atoms += f"{elmt} {n*a+shift:.5f} 0.00000 0.00000 \n"

# atoms = '''
# O  -1.551007  -0.114520   0.000000
# H  -1.934259   0.762503   0.000000
# H  -0.599677   0.040712   0.000000
# O   1.350625   0.111469   0.000000
# H   1.680398  -0.373741  -0.758561
# H   1.680398  -0.373741   0.758561
# '''
# basis = 'ccpvdz'

mol = gto.M(atom=atoms, basis=basis, spin=0, verbose=4, max_memory=16000)
mf = scf.UHF(mol).density_fit()
mf.kernel()

nfrozen = 0

from pyscf import lo
import numpy as np
orbocca = mf.mo_coeff[0][:,nfrozen:np.count_nonzero(mf.mo_occ[0])]
orbloca = lo.PipekMezey(mol, orbocca).kernel()
orboccb = mf.mo_coeff[1][:,nfrozen:np.count_nonzero(mf.mo_occ[1])]
orblocb = lo.PipekMezey(mol, orboccb).kernel()
lo_coeff = [orbloca, orblocb]

oa = [[[i],[]] for i in range(orbloca.shape[1])]
ob = [[[],[i]] for i in range(orblocb.shape[1])]
frag_lolist = oa + ob

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#7309-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Mon Dec 29 21:06:15 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 10
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.

In [3]:
from collections.abc import Iterable
from pyscf.lno import ulnoccsd
from ad_afqmc.lno_afqmc import ulno_afqmc

thresh = 3e-4
run_frg_list = [0]
chol_cut = 1e-5

mlno = ulnoccsd.ULNOCCSD(mf, lo_coeff, frag_lolist, frozen=nfrozen).set(verbose=4)
mlno.lno_thresh = [thresh*10,thresh]
lno_thresh = mlno.lno_thresh
lno_type = ['1h','1h']
lno_thresh = [1e-5, 1e-6] if lno_thresh is None else lno_thresh
print(lno_thresh)
lno_pct_occ = None
lno_norb = None
lo_proj_thresh = 1e-10
lo_proj_thresh_active = 0.1
eris = None

if run_frg_list is not None:
    frag_lolist = [frag_lolist[i] for i in run_frg_list]

nfrag = len(frag_lolist)
if lno_pct_occ is None:
    lno_pct_occ = [None, None]
if lno_norb is None:
    lno_norb = [[None,None]] * nfrag
mf = mlno._scf

if eris is None: eris = mlno.ao2mo()

# Loop over fragment
# frag_res = [None] * nfrag
for ifrag, loidx in enumerate(frag_lolist):
    if len(loidx) == 2 and isinstance(loidx[0], Iterable): # Unrestricted
        orbloc = [lo_coeff[0][:,loidx[0]], lo_coeff[1][:,loidx[1]]]
        lno_param = [
            [
                {
                    'thresh': (
                        lno_thresh[i][s] if isinstance(lno_thresh[i], Iterable)
                        else lno_thresh[i]
                    ),
                    'pct_occ': (
                        lno_pct_occ[i][s] if isinstance(lno_pct_occ[i], Iterable)
                        else lno_pct_occ[i]
                    ),
                    'norb': (
                        lno_norb[ifrag][i][s] if isinstance(lno_norb[ifrag][i], Iterable)
                        else lno_norb[ifrag][i]
                    ),
                } for i in [0, 1]
            ] for s in range(2)
        ]

    else:
        orbloc = lo_coeff[:,loidx]
        lno_param = [{'thresh': lno_thresh[i], 'pct_occ': lno_pct_occ[i],
                        'norb': lno_norb[ifrag][i]} for i in [0,1]]

    lno_coeff, lno_frozen, uocc_loc, frag_msg = mlno.make_las(eris, orbloc, lno_type, lno_param)
    # frag_res[ifrag], frag_msg = mlno.impurity_solve(mf, mo_coeff, uocc_loc, eris, frozen=frozen)
        
        # return (emp2,eccsd,ept)
        # run_afqmc(options,nproc)
        # os.system(f'mv afqmc.out lnoafqmc.out{ifrag+1}')


[0.0029999999999999996, 0.0003]
LO occ proj: 1 active | 0 standby | 4 orthogonal
LO occ proj: 0 active | 0 standby | 5 orthogonal


In [4]:
lno_frozen

[array([0, 1, 2, 8, 9]), array([0, 1, 2, 3, 8, 9])]

In [5]:
prja = uocc_loc[0] @ uocc_loc[0].T.conj()
prjb = uocc_loc[1] @ uocc_loc[1].T.conj()
prjlo = [prja, prjb]

In [6]:
mol = mf.mol
nocc_a = int(sum(mf.mo_occ[0]))
actfrag_a = np.array([i for i in range(mol.nao) if i not in lno_frozen[0]])
frzocc_a = np.array([i for i in range(nocc_a) if i in lno_frozen[0]])
actocc_a = np.array([i for i in range(nocc_a) if i in actfrag_a])
actvir_a = np.array([i for i in range(nocc_a,mol.nao) if i in actfrag_a])
nfrzocc_a = len(frzocc_a)
nactocc_a = len(actocc_a)
nactvir_a = len(actvir_a)
nactorb_a = len(actfrag_a)
nocc_b = int(sum(mf.mo_occ[1]))
actfrag_b = np.array([i for i in range(mol.nao) if i not in lno_frozen[1]])
frzocc_b = np.array([i for i in range(nocc_b) if i in lno_frozen[1]])
actocc_b = np.array([i for i in range(nocc_b) if i in actfrag_b])
actvir_b = np.array([i for i in range(nocc_b,mol.nao) if i in actfrag_b])
nfrzocc_b = len(frzocc_b)
nactocc_b = len(actocc_b)
nactvir_b = len(actvir_b)
nactorb_b = len(actfrag_b)

ncas = (nactorb_a, nactorb_b)
ncore = (nfrzocc_a, nfrzocc_b)
nelec = (nactocc_a, nactocc_b)
print(ncas)
print(ncore)
print(nelec)

(5, 4)
(3, 4)
(2, 1)


In [10]:
mo_occ = mlno.mo_occ
mlno.verbose_imp = 4
lno_frozen, maskact = ulnoccsd.get_maskact(lno_frozen, [mo_occ[0].size, mo_occ[1].size])
mcc = ulnoccsd.UCCSD(mf, mo_coeff=lno_coeff, frozen=lno_frozen).set(verbose=mlno.verbose_imp)
mcc._s1e = mlno._s1e
mcc._h1e = mlno._h1e
mcc._vhf = mlno._vhf
if mlno.kwargs_imp is not None:
    mcc = mcc.set(**mlno.kwargs_imp)

(emp2,eccsd,ept), t1, t2 =\
    ulno_afqmc.ulno_ccsd(mcc, lno_coeff, uocc_loc, mo_occ, maskact, ccsd_t=True)
print(emp2,eccsd,ept)


WARN: CCSD detected DF being used in the HF object. MO integrals are computed based on the DF 3-index tensors.
It's recommended to use dfccsd.CCSD for the DF-CCSD calculations

Init t2, MP2 energy = -0.0755731888275522

******** <class 'pyscf.lno.ulnoccsd.MODIFIED_UCCSD'> ********
CC2 = 0
CCSD nocc = (2, 1), nmo = (5, 4)
frozen orbitals [array([0, 1, 2, 8, 9]), array([0, 1, 2, 3, 8, 9])]
max_cycle = 50
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-06
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 16000 MB (current use 259 MB)
Init E_corr(MODIFIED_UCCSD) = -0.0755731888275522
cycle = 1  E_corr(MODIFIED_UCCSD) = -0.115364252521905  dE = -0.0397910637  norm(t1,t2) = 0.187459
cycle = 2  E_corr(MODIFIED_UCCSD) = -0.133975010881738  dE = -0.0186107584  norm(t1,t2) = 0.0935963
cycle = 3  E_corr(MODIFIED_UCCSD) = -0.144591464328145  dE = -0.0106164534  norm(t1,t2) = 0.0372402
cycle = 4  E_corr(MODIFIED_UCCSD) = -0.145610747427725  dE = -0.0010192831  norm(t1,t

In [12]:
options = {'n_eql': 5,
        'n_prop_steps': 50,
        'n_ene_blocks': 1,
        'n_sr_blocks': 10,
        'n_blocks': 10,
        'n_walkers': 20,
        'n_batch': 2,
        'seed': 98,
        'walker_type': 'uhf',
        'trial': 'uccsd_pt2',
        'dt':0.005,
        'free_projection':False,
        'ad_mode':None,
        'use_gpu': False,
        'max_error': 1e-4
        }

from mpi4py import MPI
if not MPI.Is_finalized():
    MPI.Finalize()

nelec, ncas = ulno_afqmc.prep_afqmc(
    mf,lno_coeff,t1,t2,lno_frozen,prjlo,
    options,chol_cut=chol_cut)

print(nelec, ncas)

# Calculating Effective Active Space One-electron Integrals
# Generating Cholesky Integrals
# DF Tensor shape: (90, 10, 10)
# Constracting cLAS that span both Alpha and Beta active space
# Naive Common LAS Shape:  (10, 9)
# Common Active Space SVD Singular values:
[1.41421356e+00 1.41419237e+00 1.41339996e+00 1.41204140e+00
 1.00000000e+00 7.83523794e-02 4.79639529e-02 7.74129151e-03
 5.06063224e-07]
# cLAS projection torr: 1e-10
# cLAS projection loss: (2.49e-02, 2.49e-02)
# cLAS projection loss: (1.36e-02, 1.39e-02)
# cLAS projection loss: (2.96e-03, 2.96e-03)
# cLAS projection loss: (1.21e-07, 1.23e-07)
# cLAS projection loss: (1.61e-15, 2.05e-15)
# Minimum size of cLAS to span both Alpha and Beta LAS: 9
# True Common LAS Shape:  (10, 9)
# Composing AO ERIs from DF basis


# Finished Composing LAS ERIs
# Decomposing MO ERIs to Cholesky vectors
# Tighter Chol_cutoff is recommended for LNO
# Cholesky cutoff is: 1e-05
# Alpha chol1 shape: (19, 5, 5)
#  Beta chol1 shape: (19, 4, 4)
# Finished calculating Cholesky integrals
# Size of the correlation space
# Number of electrons: (2, 1)
# Number of basis functions: (5, 4)
# Alpha Basis Cholesky shape: (19, 5, 5)
#  Beta Basis Cholesky shape: (19, 4, 4)
(2, 1) (5, 4)


In [7]:
from ad_afqmc import config
config.setup_jax()
MPI = config.setup_comm()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
from ad_afqmc.lno_afqmc import ulno_afqmc
ham_data, prop, trial, wave_data, sampler, options, _ = (ulno_afqmc._prep_afqmc())

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: (5, 4)
# nelec: (2, 1)
#
# n_eql: 5
# n_prop_steps: 50
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 20
# n_batch: 2
# seed: 98
# walker_type: uhf
# trial: uccsd_pt2
# dt: 0.005
# free_projection: False
# use_gpu: False
# max_error: 0.0001
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
#


In [10]:
import time
from jax import numpy as jnp
from jax import random
init_time = time.time()
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

### initialize propagation
seed = options["seed"]
init_walkers = None
# dm_up = jnp.array(wave_data["mo_coeff"][0] @ wave_data["mo_coeff"][0].T.conj())
# dm_dn = jnp.array(wave_data["mo_coeff"][1] @ wave_data["mo_coeff"][1].T.conj())
# trial_rdm1 = [dm_up, dm_dn]
trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
ham_data = trial._build_measurement_intermediates(ham_data, wave_data)
ham_data = prop._build_propagation_intermediates(ham_data, trial, wave_data)

prop_data = prop.init_prop_data(trial, wave_data, ham_data, init_walkers)
if jnp.abs(jnp.sum(prop_data["overlaps"])) < 1.0e-6:
    raise ValueError(
        "Initial overlaps are zero. Pass walkers with non-zero overlap."
    )
prop_data["key"] = random.PRNGKey(seed + rank)

prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
prop_data["n_killed_walkers"] = 0
prop_data["pop_control_ene_shift"] = prop_data["e_estimate"]
print(prop_data["e_estimate"])
# E(MODIFIED_UCCSD) = -3.085067447898267  E_corr = -0.01684699454462395
# LNO-UCCSD (T1 = 0) fragment energy: -0.008283544474655563


-4.0007863898965


In [11]:
walker_up, walker_dn = prop_data['walkers'][0][0], prop_data['walkers'][1][0]
e0, t1olp, eorb_bar, t2eorb, t2orb, e12bar = trial._calc_eorb_pt2(walker_up, walker_dn,ham_data,wave_data)
print(e0, t1olp, eorb_bar, t2eorb, t2orb, e12bar)

(-4.000786389896499+0j) (1+0j) (-3.3704230314404024e-05+0j) (-0.071089153060341+0j) 0j (-1.6075857741933324+0j)


In [12]:
from jax import lax, vmap
walkers = prop_data['walkers']
n_walkers = walkers[0].shape[0]
batch_size = n_walkers // trial.n_batch

def scan_batch(carry, walker_batch):
    batch_walker_up, batch_walker_dn = walker_batch
    e0, t1olp, eorb_bar, t2eorb, t2orb, e12bar \
        = vmap(trial._calc_eorb_pt2, in_axes=(0, 0, None, None))(
        batch_walker_up, batch_walker_dn, ham_data, wave_data
    )
    return carry, (e0, t1olp, eorb_bar, t2eorb, t2orb, e12bar)

_, (e0, t1olp, eorb_bar, t2eorb, t2orb, e12bar) \
    = lax.scan(scan_batch, None,
    (
        walkers[0].reshape(trial.n_batch, batch_size, trial.norb[0], trial.nelec[0]),
        walkers[1].reshape(trial.n_batch, batch_size, trial.norb[1], trial.nelec[1]),
    ),
)

In [18]:
print(e0.reshape(n_walkers))

[-4.00078639+0.j -4.00078639+0.j -4.00078639+0.j -4.00078639+0.j
 -4.00078639+0.j -4.00078639+0.j -4.00078639+0.j -4.00078639+0.j
 -4.00078639+0.j -4.00078639+0.j -4.00078639+0.j -4.00078639+0.j
 -4.00078639+0.j -4.00078639+0.j -4.00078639+0.j -4.00078639+0.j
 -4.00078639+0.j -4.00078639+0.j -4.00078639+0.j -4.00078639+0.j]


In [16]:
print((e0, t1olp, eorb_bar, t2eorb, t2orb, e12bar).shape)

AttributeError: 'tuple' object has no attribute 'shape'